
*Technical University of Munich<br>
Professorship of Environmental Sensing and Modeling<br><br>*
**Author:**  Daniel Kühbacher<br>
**Date:**  16.01.2024

--- 

# Calculate Cold Start Emissions using HBEFA Emission factors

<!--Notebook description and usage information-->
In this notebook, HBEFA Cold-Start emission factors are applied to input from the visum model. <br>
HBEFA allows to separate cold-start emissions by: 
- Ambient temperature 
- Trip length 
- Parking hours (to determine how hot the engine is before the starting process)

HBEFA provides cold disaggregated cold start emissions where each of the parameters above can be set based on individual data. Since no Munich-specific information on trip lenght or parking hours is available, aggregated values for these categories are used. Temperature information is provided by a weather station in Munich and will be used for the whole region.<br>
It should be mentioned, that HBEFA just provides cold-start emissions for Personal Cars and Small Vans. They conistitue the main part of the overall number of cold starts in a city.


In [55]:
import sys

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# import custom modules
sys.path.append('../utils')
import data_paths
import traffic_counts
from hbefa_cold_emissions import HbefaColdEmissions

from lmu_meteo_api import interface
from datetime import datetime

## Import data

In [48]:
# import visum O-D matricies
visum_links = gpd.read_file(data_paths.VISUM_FOLDER_PATH + "visum_links.GPKG",
                            driver = 'GPKG')

# caclulate starts per squaremeter before gridding
visum_links['PC_starts_per_meter'] = visum_links['PC_cold_starts'] / visum_links['geometry'].length
visum_links['LCV_starts_per_meter'] = visum_links['LCV_cold_starts'] / visum_links['geometry'].length


munich_boarders = gpd.read_file(data_paths.MUNICH_BOARDERS_FILE).to_crs(25832)
visum_links_clipped = gpd.clip(visum_links, munich_boarders)
visum_links_clipped = visum_links_clipped.explode(ignore_index=True) # convert multipolygons to polygons

## Notebook functions

In [49]:
def annual_temperature_profile(start_date:datetime, 
                               end_date:datetime,
                               aggregate = 'H') -> pd.Series:
    """Downloads meteo data from the LMU Meteo station and returns a dataframe 
    with hourly temperatures for Munich

    Args:
        year (int): Year
        aggregate (str, optional): aggregate to specified timeframe. Defaults to 'H'.

    Returns:
        pd.Series: temperature profile
    """
    
    start_time = start_date.strftime('%Y-%m-%d') + 'T00-00-00'
    end_time = end_date.strftime('%Y-%m-%d') + 'T23-59-59'
    
    if datetime.strptime(end_time, '%Y-%m-%dT%H-%M-%S').date() > datetime.now().date():
        end_time = datetime.now().strftime('%Y-%m-%dT%H-%M-%S')
    
    lmu_api = interface.meteo_data()
    data = lmu_api.get_meteo_data(parameters = ["air_temperature_2m"], 
                                station_id = 'MIM01', 
                                start_time = start_time, 
                                end_time = end_time)
    
    return (data.air_temperature_2m - 273.15).resample(aggregate).mean()

## Initialize objects and download temperature data

In [50]:
# import trafic datasets
cycles = traffic_counts.TrafficCounts()

# download temperature data for 2019
temperature = annual_temperature_profile(start_date=datetime(2019,1,1),
                                         end_date=datetime(2022,12,31)) 

# instanciate cold start emission object
cs_obj = HbefaColdEmissions()


Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_ColdStart_Vehcat_ColdStart.XLS


In [51]:
parameters = pd.DataFrame(index = pd.date_range(start='2019-01-01 00:00:00',
                                                end = '2022-12-31 23:00:00',
                                                freq='1h'))

reference_scaling_road_class = 'Distributor/Secondary'

parameters['temperature'] = temperature
parameters['hour_factor_PC'] = cycles.timeprofile[reference_scaling_road_class]['PC']
parameters['hour_factor_LCV'] = cycles.timeprofile[reference_scaling_road_class]['LCV']

## Calculate total emissions for Munich

In [52]:
#calculate daily coldstarts in Munich
daily_PC_starts = (visum_links_clipped['PC_starts_per_meter'] * visum_links_clipped.length).sum()
daily_LCV_starts = (visum_links_clipped['LCV_starts_per_meter'] * visum_links_clipped.length).sum()

em_list_pc = list()
em_list_lcv = list()

PC_result = pd.DataFrame()
LCV_result = pd.DataFrame()

for idx, row in parameters.iterrows():
    
    # get emission factors
    em_PC = cs_obj.calculate_emission_daily(hourly_temperature=row['temperature'],
                                            vehicle_class='PC',
                                            year = idx.year)
    
    em_LCV = cs_obj.calculate_emission_daily(hourly_temperature=row['temperature'],
                                             vehicle_class='LCV',
                                             year = idx.year)
    
    # hourly number of starting procedures
    hourly_PC_starts = daily_PC_starts * row['hour_factor_PC']
    hourly_LCV_starts = daily_LCV_starts * row['hour_factor_LCV']
    
    PC_result = pd.concat([PC_result, (em_PC * hourly_PC_starts)], axis=1)
    LCV_result = pd.concat([LCV_result, (em_LCV * hourly_LCV_starts)], axis=1)
    
PC_result = PC_result.transpose().set_index(pd.date_range(start='2019-01-01 00:00:00',
                                                end = '2022-12-31 23:00:00',
                                                freq='1h'))    
LCV_result = LCV_result.transpose().set_index(pd.date_range(start='2019-01-01 00:00:00',
                                                end = '2022-12-31 23:00:00',
                                                freq='1h'))   

In [106]:
result = PC_result.copy()
result['vehicle_class'] = 'PC'
result = pd.concat([result, LCV_result], axis = 0)
result.loc[result['vehicle_class'].isna(), 'vehicle_class'] = 'LCV'

result_agg = result.groupby(['vehicle_class']).resample('1Y').sum(numeric_only = True)

columns_to_keep = ['CO', 'NOx', 'PM', 'CO2(rep)', 'CO2(total)', 
                  'NO2', 'CH4', 'BC (exhaust)', 'CO2e']

result_agg = result_agg[columns_to_keep]